In [5]:
%run ../src/parsing/preset.py

In [6]:
res = make_subj_pred(7, 'w', "../data/feature_data.gzip", "../models/XGBoost", "Южно-Якутский энергорайон")

In [7]:
res.head(5)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область,lag_hour,lag_day,lag_week,lag_month,lag_year,actual_consumption
5500725,114146.0,2024.0,5.0,9.0,0.0,4.0,130.0,19.0,3.0,1.0,...,0.0,0.0,1.0,0.0,378.0,354.0,396.0,414.0,420.0,NaN
5500800,114147.0,2024.0,5.0,9.0,1.0,4.0,130.0,19.0,3.0,1.0,...,0.0,0.0,1.0,0.0,382.0,345.0,388.0,401.0,424.0,NaN
5500875,114148.0,2024.0,5.0,9.0,2.0,4.0,130.0,19.0,3.0,1.0,...,0.0,0.0,1.0,0.0,374.0,346.0,398.0,400.0,422.0,NaN
5500950,114149.0,2024.0,5.0,9.0,3.0,4.0,130.0,19.0,3.0,1.0,...,0.0,0.0,1.0,0.0,380.0,343.0,392.0,391.0,419.0,NaN
5501025,114150.0,2024.0,5.0,9.0,4.0,4.0,130.0,19.0,3.0,1.0,...,0.0,0.0,1.0,0.0,388.0,344.0,401.0,393.0,405.0,NaN
